In [55]:
#######################################
#
#   Multilayer Perceptron for MNIST 
#
#######################################

In [56]:
#import libraries
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib

In [57]:
#set variables
weights = []
bias = []
layers_config = [784, 512, 10]
average_weights = []
average_bias = []
eta = 0.01
epoch_accuracy = []

In [58]:
# some functions
def ReLU(x):
    return np.maximum(0, x)

def ReLU_derivative(values):
    result = [1 if x > 0 else 0 for x in values]
    return result

def tanh_activation(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - (np.tanh(x) ** 2)

def Softmax(x):
    return np.exp(x - np.max(x))/(np.sum(np.exp(x - np.max(x))))

In [59]:
def feedforward(input_image):
    a = []
    a.append(input_image.reshape(len(input_image[0])*len(input_image[1]),1))
    
    for i in range(1, len(layers_config)-1):
        a.append(ReLU((weights[i] @ a[i-1]).reshape(len(weights[i]), 1) + bias[i]))
    y_hat = Softmax((weights[-1] @ a[-1]) + bias[-1]).reshape(len(weights[-1]),)
    a.append(y_hat)
    
    return a

In [60]:
def backprop(a, ground_output_y):
    delta_error = list(np.empty_like(a))
    index_count = len(layers_config) - 1
    delta_error[index_count] = (a[index_count] - ground_output_y).reshape(len(a[index_count]), 1)
    average_bias[index_count] = average_bias[index_count] + delta_error[index_count] # Output Layer
    average_weights[index_count] = average_weights[index_count] + (delta_error[index_count] @ a[index_count - 1].T) # Output Layer
    for i in range(index_count - 1, 0, -1):
        h_derivative = np.array(ReLU_derivative(a[i])).reshape(1, len(a[i])) * np.eye(len(a[i]))
        delta_error[i] = h_derivative.T @ weights[i+1].T @ delta_error[i+1]
        average_bias[i] = average_bias[i] + delta_error[i]
        average_weights[i] = average_weights[i] + (delta_error[i] @ a[i-1].T)

In [61]:
# He Normalization
def initialize_weights():
    if len(layers_config) < 3:
        print("Incorrect network structure. Check the neural network layer configuration")
    else:
        layer_count = len(layers_config)
        weights.append([])
        bias.append([])
        average_weights.append([])
        average_bias.append([])
        for i in range(1, layer_count):
            neurons_previous = layers_config[i-1]
            neurons_current = layers_config[i]
            single_layer_weights = np.random.normal(0, np.sqrt(2/neurons_previous), (neurons_current, neurons_previous))
            single_layer_bias = np.random.normal(0, np.sqrt(2/neurons_previous), (neurons_current, 1))
            weights.append(single_layer_weights)
            bias.append(single_layer_bias)
            average_weights.append(single_layer_weights)
            average_bias.append(single_layer_bias)

In [62]:
initialize_weights()

In [63]:
#load data
data_path = "mnist_traindata.hdf5"
with h5py.File(data_path, 'r') as hf:
    train_x, train_y = hf['image'][...], hf['label'][...]
    
# split into training and validation data
X_train, X_val, y_train, y_val = train_test_split(xdata, ydata, test_size=500, random_state=1)

epoch_size = 1
batch_size = 500
batch_numbers = int(len(X_train)/batch_size)

In [54]:
#Single Sample Updates
shuffle_order = np.random.permutation(len(X_train))
for i in range(1, epoch_size + 1):
    print("Running Epoch {}".format(i))
    if i == 2:
        eta = eta/2
    if i == 3:
        eta = eta/2
    shuffle_order = np.random.permutation(len(X_train))
    for j in range(batch_numbers):
        print("Running batch number {} in epoch {}".format(j, i))
        for k in range(batch_size):
            shuffle_index = j * batch_size + k
            sample_x = X_train[shuffle_index]
            sample_y = y_train[shuffle_index]
            a_values = feedforward(sample_x)
            backprop(a_values, sample_y)
        for a in range(len(weights)):
            value_weight = weights[a]
            value_average_weight = np.multiply(average_weights[a], (eta/batch_size))
            weights[a] = np.subtract(value_weight, value_average_weight)
            value_bias = bias[a]
            value_average_bias = np.multiply(average_bias[a], (eta/batch_size))
            bias[a] = np.subtract(value_bias, value_average_bias)
    print("Running feedforward on validation data for epoch {}".format(i))
    y_output = np.array([feedforward(X_val[m, :])[len(layers_config) - 1] for m in range(len(X_val))])
    class_output = np.argmax(y_output, axis=0)
    label_class = np.argmax(y_val, axis=0)
    number_correct_classification = np.sum(class_output == label_class)
    accuracy_val = number_correct_classification / len(X_val)
    epoch_accuracy.append(accuracy_val)
    print("Accuracy on Validation Set for epoch {} is {}".format(epoch_size, accuracy_val))

plt.plot(range(1, epoch_size + 1), epoch_accuracy)
plt.title('Model accuracy after each epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.axvline(x=1)
plt.show()

Running Epoch 1
Running batch number 0 in epoch 1
Running batch number 1 in epoch 1
Running batch number 2 in epoch 1
Running batch number 3 in epoch 1
Running batch number 4 in epoch 1
Running batch number 5 in epoch 1
Running batch number 6 in epoch 1
Running batch number 7 in epoch 1
Running batch number 8 in epoch 1
Running batch number 9 in epoch 1
Running batch number 10 in epoch 1
Running batch number 11 in epoch 1
Running batch number 12 in epoch 1
Running batch number 13 in epoch 1
Running batch number 14 in epoch 1
Running batch number 15 in epoch 1
Running batch number 16 in epoch 1
Running batch number 17 in epoch 1
Running batch number 18 in epoch 1
Running batch number 19 in epoch 1
Running batch number 20 in epoch 1
Running batch number 21 in epoch 1
Running batch number 22 in epoch 1
Running batch number 23 in epoch 1
Running batch number 24 in epoch 1
Running batch number 25 in epoch 1
Running batch number 26 in epoch 1
Running batch number 27 in epoch 1
Running batch 

/home/hasnain/wd/ec504project/working/ml/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in add
  
/home/hasnain/wd/ec504project/working/ml/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in add
  # This is added back by InteractiveShellApp.init_path()


Running batch number 31 in epoch 1


/home/hasnain/wd/ec504project/working/ml/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in matmul
  if __name__ == '__main__':
/home/hasnain/wd/ec504project/working/ml/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in matmul
  import sys


Running batch number 32 in epoch 1
Running batch number 33 in epoch 1
Running batch number 34 in epoch 1


KeyboardInterrupt: 

In [52]:
class_output = np.argmax(y_output, axis=0)
label_class = np.argmax(y_val, axis=0)
number_correct_classification = np.sum(class_output == label_class)
accuracy_val = number_correct_classification / len(X_val)
epoch_accuracy.append(accuracy_val)
print("Accuracy on Validation Set for epoch {} is {}".format(epoch_size, accuracy_val))

#plt.plot(range(1, epoch_size + 1), epoch_accuracy)
#plt.title('Model accuracy after each epoch')
#plt.ylabel('Accuracy')
#plt.xlabel('Epoch')
#plt.axvline(x=1)
#plt.show()

Accuracy on Validation Set for epoch 1 is 0.0


In [ ]:

# Training Set Accuracy
y_output = np.array([feedforward(X_train[i, :])[len(layers_config) - 1] for i in range(len(X_train))])
class_output = np.argmax(y_output, axis=1)
label_class = np.argmax(y_train, axis = 1)
number_correct_classification = np.sum(class_output == label_class)
accuracy_train = number_correct_classification/len(X_train)
print("Accuracy on Training Set is {}".format(accuracy_train))


# Validation Set Accuracy
y_output = np.array([feedforward(X_val[i, :])[len(layers_config) - 1] for i in range(len(X_val))])
class_output = np.argmax(y_output, axis=1)
label_class = np.argmax(y_val, axis = 1)
number_correct_classification = np.sum(class_output == label_class)
accuracy_val = number_correct_classification/len(X_val)
print("Accuracy on Test Set is {}".format(accuracy_val))

In [ ]:
input_image